In [1]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

In [2]:
spacex_df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv', index_col=False)
spacex_df.drop(columns='Unnamed: 0', inplace=True)
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
spacex_df.shape

(56, 6)

In [5]:
# Create a dash application
from cProfile import label
from numpy import float64


app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             style={'font-size': 20},
                                             options=[
                                                 {'label': 'All Sites',
                                                  'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40',
                                                     'value': 'CCAFS LC-40'},
                                                 {'label': 'VAFB SLC-4E',
                                                     'value': 'VAFB SLC-4E'},
                                                 {'label': 'KSC LC-39A',
                                                     'value': 'KSC LC-39A'},
                                                 {'label': 'CCAFS SLC-40',
                                                  'value': 'CCAFS SLC-40'},
                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),

                                html.Br(),

                                html.P("Payload range (Kg):",
                                       style={'font-size': 20}),

                                # TASK 3: Add a slider to select payload range
                                # dcc.RangeSlider(id='payload-slider',...)

                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: {'label': '0', 'style': {'font-size': 23}},
                                                       2500: {'label': '2,500', 'style': {'font-size': 23}},
                                                       5000: {'label': '5,000', 'style': {'font-size': 23}},
                                                       7500: {'label': '7,500', 'style': {'font-size': 23}},
                                                       10000: {'label': '10,000', 'style': {'font-size': 23}},
                                                       },
                                                value=[min_payload, max_payload]),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(
                                    dcc.Graph(
                                        id='success-payload-scatter-chart')
),
])

# # Function decorator to specify function input and output


@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        spacex_df_class = spacex_df.groupby(
            ['class'])['Launch Site'].count().reset_index()
        fig = px.pie(spacex_df_class, values='Launch Site',
                     names=['Failure', 'Success'],
                     color=['Failure', 'Success'],
                     color_discrete_map={'Success': 'blue',
                                         'Failure': 'red'},
                     title='Total Success Launches By Site'
                     )
        fig.update_traces(textposition='inside',
                          textinfo='percent+label', textfont_size=20)
        fig.update_layout(showlegend=False, margin=dict(t=0, b=0, l=0, r=0))
        return fig
    else:
        # return the outcomes piechart for a selected site
        spacex_df_class_ent = spacex_df[spacex_df['Launch Site'] == entered_site].groupby(
            ['class'])['Launch Site'].count().reset_index()
        fig = px.pie(spacex_df_class_ent, values='Launch Site',
                     names=['Failure', 'Success'],
                     color=['Failure', 'Success'],
                     color_discrete_map={'Success': 'blue',
                                         'Failure': 'red'},
                     title='Total Success Launches By Site'
                     )
        fig.update_traces(textposition='inside',
                          textinfo='percent+label', textfont_size=20)
        fig.update_layout(showlegend=False, margin=dict(t=0, b=0, l=0, r=0))
        return fig


@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property='value')])
def get_scatter_chart(entered_site, payload):
    if entered_site == 'ALL':
        spacex_df_sc = spacex_df[['Payload Mass (kg)', 'class', 'Booster Version Category', 'Launch Site']][(
            spacex_df['Payload Mass (kg)'] <= payload[1]) & (spacex_df['Payload Mass (kg)'] >= payload[0])]
        fig_s = px.scatter(spacex_df_sc,
                           x='Payload Mass (kg)',
                           y='class',
                           color='Booster Version Category',
                           title='Correlation Between Payload and Success for All Sites'
                           )
        fig_s.update_yaxes(dtick=1)
        fig_s.update_xaxes(dtick=1000)
        fig_s.update_layout(
            font_size=23,
            title_font_size=23)
        fig_s.update_traces(marker=dict(size=12,
                                        line=dict(width=2,
                                                  color='DarkSlateGrey')),
                            selector=dict(mode='markers'))
        return fig_s
    else:
        spacex_df_sc = spacex_df[['Payload Mass (kg)', 'class', 'Booster Version Category', 'Launch Site']][(
            spacex_df['Payload Mass (kg)'] <= payload[1]) & (spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Launch Site'] == entered_site)]
        fig_s = px.scatter(spacex_df_sc,
                           x='Payload Mass (kg)',
                           y='class',
                           color='Booster Version Category',
                           title=f'Correlation Between Payload and Success for {entered_site}'
                           )
        fig_s.update_yaxes(dtick=1)
        fig_s.update_xaxes(dtick=1000)
        fig_s.update_layout(
            font_size=23,
            title_font_size=23)
        fig_s.update_traces(marker=dict(size=12,
                                        line=dict(width=2,
                                                  color='DarkSlateGrey')),
                            selector=dict(mode='markers'))
        return fig_s


In [6]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [20/Jul/2022 14:59:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 14:59:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 14:59:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 14:59:58] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Jul/2022 14:59:58] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Jul/2022 14:59:58] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Jul/2022 14:59:58] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Jul/2022 14:59:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 15:00:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 15:00:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 15:00:43] "POST /_dash-u